# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [9]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import NonSequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

# Define architectures

Define each one of the different architectures to be tested.

In [3]:
#Clear the previous tensorflow graph
K.clear_session()
lambda_regularization = 0.20

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model



# Process Data

In [4]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 128

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

init
super init


# Build the model

In [6]:
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = RULmodel_SN_5(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = NonSequenceTunableModelRegression('ModelRUL_SN_5', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps, 
                                 data_scaler=min_max_scaler)

# Test on dataset 1

In [11]:
tModel.load_data(verbose=1, cross_validation_ratio=0)


Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file
Epoch 1/100
17731/17731 [==============================] - 1s 40us/step - loss: 7049.5075 - mean_squared_error: 7041.5567
Epoch 2/100
17731/17731 [==============================] - 0s 9us/step - loss: 4625.9737 - mean_squared_error: 4615.8924

# Print Out Result

In [15]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, setLimits=[7,145])
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 243us/step
scores
Engine 1, Predicted RUL [114.19871], Rounded RUL 114.0, Real RUL 112.0
Engine 2, Predicted RUL [119.207054], Rounded RUL 119.0, Real RUL 98.0
Engine 3, Predicted RUL [61.742973], Rounded RUL 61.0, Real RUL 69.0
Engine 4, Predicted RUL [86.28421], Rounded RUL 86.0, Real RUL 82.0
Engine 5, Predicted RUL [99.85318], Rounded RUL 99.0, Real RUL 91.0
Engine 6, Predicted RUL [104.48811], Rounded RUL 104.0, Real RUL 93.0
Engine 7, Predicted RUL [105.924736], Rounded RUL 105.0, Real RUL 91.0
Engine 8, Predicted RUL [90.76726], Rounded RUL 90.0, Real RUL 95.0
Engine 9, Predicted RUL [117.63211], Rounded RUL 117.0, Real RUL 111.0
Engine 10, Predicted RUL [90.05551], Rounded RUL 90.0, Real RUL 96.0
Engine 11, Predicted RUL [89.2202], Rounded RUL 89.0, Real RUL 97.0
Engine 12, Predicted RUL [101.87789], Rounded RUL 101.0, Real RUL 124.0
Engine 13, Predicted RUL [84.79732], Rounded RUL 84.0, R

TypeError: zip argument #1 must support iteration

# Test on all datasets

Test the choosen model on the four datasets.

In [15]:
datasets = [1,2,3,4]
iterations = 10
tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
windowSizes = {1:18,2:18,3:18,4:18}
strides = {1:2,2:2,3:2,4:2}
constRUL = {1:128, 2:128, 3:128, 4:128}
#models = {1:RULmodel_SN_1,2:RULmodel_SN_2,3:RULmodel_SN_3,4:RULmodel_SN_4, 5:RULmodel_SN_5, 6:RULmodel_SN_6}
models = {1:RULmodel_SN_5}

file = open("ResultsDatasets_NonRectified_NoLimits_smallerMLP2_2.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

for key, model in models.items():
    
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
    
    for i in range(1,5):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')

        #Create and compile the models
        windowSize = windowSizes[i]
        nFeatures = len(selected_features)
        shapeSN = nFeatures*windowSize
        modelRULSN = model(shapeSN)
        modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

        tModel.changeModel('ModelRUL_SN_Dat'+str(shapeSN), modelRULSN, 'keras')
        tModel.windowSize = windowSize
        tModel.windowStride = strides[i]
        tModel.constRul = constRUL[i]
        tModel.changeDataset(dataset)
        tModel.loadData(verbose=1, rectify_labels = False)
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))

        for j in range(iterations):

            print("Iteration "+str(j+1))
            #file.write("Iteration "+str(j+1)+'\n\n')
            tModel.trainModel(learningRateScheduler=lrate, verbose=0)
            #tModel.evaluateModel(["rhs"], round=2, setLimits=[7,135])
            tModel.evaluateModel(["rhs"], round=2)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.trainTime

            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time

        #print(tempScoresRMSE)
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
        #file.write(str(stats.describe(tempScoresRMSE))+'\n\n')
        #file.write(str(stats.describe(tempScoresRHS))+'\n\n')
        #file.write(str(stats.describe(tempTime))+'\n\n')
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
        
file.close()

For model 1
Computing for dataset 1
Loading data for dataset 1 with window_size of 18, stride of 2 and constRUL of 128. Cros-Validation ratio 0
Data loaded for dataset 1
Iteration 1
100/100 [==============================] - 0s 1ms/step
Iteration 2
100/100 [==============================] - 0s 61us/step
Iteration 3
100/100 [==============================] - 0s 40us/step
Iteration 4
100/100 [==============================] - 0s 40us/step
Iteration 5
100/100 [==============================] - 0s 50us/step
Iteration 6
100/100 [==============================] - 0s 30us/step
Iteration 7
100/100 [==============================] - 0s 40us/step
Iteration 8
100/100 [==============================] - 0s 40us/step
Iteration 9
100/100 [==============================] - 0s 40us/step
Iteration 10
100/100 [==============================] - 0s 40us/step
DescribeResult(nobs=10, minmax=(17.588916964952674, 18.56932955170972), mean=17.820665657547316, variance=0.08430613545967351, skewness=1.889559813241

# Save Best Model

Save the model and weights of the best model throughout 10 iterations

In [3]:
#Clear the previous tensorflow graph
K.clear_session()

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

dataFolder = '../CMAPSSData'

datasets = [1]
iterations = 10
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
#windowSizes = {1:25,2:15,3:25,4:15}
windowSizes = 15
stride = 2
#constRUL = {1:93, 2:94, 3:92, 4:94}
constRUL = 95
#models = {1:RULmodel_SN_1,2:RULmodel_SN_2,3:RULmodel_SN_3,4:RULmodel_SN_4}
models = {1:RULmodel_SN_1}

#Create and compile the model
windowSize = 15
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.loadData()
tModel.epochs = 200

#file = open("ResultsBestModelAllDatasetsSingleSet.csv", "w")
#csvfile = csv.writer(file, lineterminator='\n')

min_rmse = 1000
bestModel = None
bestIndex = 0

for i in range(iterations):

    print("Iteration "+str(i+1))
    #file.write("Iteration "+str(j+1)+'\n\n')
    tModel.trainModel(learningRateScheduler=lrate, verbose=0)
    tModel.evaluateModel(["rhs"], round=True)

    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rmse2 = cScores['rmse']
    rhs = cScores['rhs']
    time = tModel.trainTime

    if rmse2 < min_rmse:
        bestModel = tModel.model
        bestIndex = i
    
    tempScoresRMSE[i] = rmse2
    tempScoresRHS[i] = rhs
    tempTime[i] = time

print(tempScoresRMSE)    
    
#save best model to file
# serialize model to JSON
model_json = bestModel.to_json()
with open("bestRULModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
bestModel.save_weights("bestRULModel.h5")
print("Saved model to disk")
        
#file.close()


Iteration 1
100/100 [==============================] - 0s 291us/step


NameError: name 'tempScoresRMSE' is not defined